In [1]:
from funcs import * # funcs include functions used across scripts
import pandas as pd
from matplotlib import pyplot as plt

read the .ict data

In [2]:
o = read_all('OPTICAL') # optical data
inl = read_all('InletFlag') # inlet flag
a = read_all('AMS') # ams
mn = read_all('MetNav') # met nav

read the DASH data

In [3]:
retrieval_paths = sorted(glob.glob('../data/v2/*/*/*.csv'))
retrieval_list = []
for i in range(0, len(retrieval_paths)):
    retrieval_list.append(pd.read_csv(retrieval_paths[i], parse_dates=['Start_Date_Time_UTC', 'Stop_Date_Time_UTC']))
r = pd.concat(retrieval_list).reset_index(drop=True)

Separate data by size

In [4]:
r_dps = []
dps = sorted(r['SEMS_Dp'].unique())
for i in range(0,len(dps)):
    r_dps.append(r[(r['RH']>75) & (r['SEMS_Dp']==dps[i])])

Rename variables so they have size in name

In [6]:
for i in range(0, len(dps)):
    d_label = str(int(dps[i]))
    r_dps[i] = r_dps[i].rename(columns={'GF_raw':'GF_raw_'+d_label})

In [13]:
def merge_df(df1, df2, st1 = 'Start_Date_Time_UTC', et1 = 'Stop_Date_Time_UTC', st2 = 'Time_Start', et2 = 'Time_Start', keys = ['GPS_Altitude']):
    out = []
    for i in range(0,len(df1)):
        row = df1.iloc[i]
        t_start, t_stop = row[st1], row[et1]
        out.append(df2[(df2[st2] >= t_start) & (df2[et2] <= t_stop)].mean(numeric_only=True))
    df_out = pd.DataFrame(out)
    df1.loc[:,keys] = df_out.loc[:,keys]
    return df1

In [15]:
d_test = merge_df(r_dps[0], mn)

In [20]:
d_test

,Start_Date_Time_UTC,Stop_Date_Time_UTC,RH,SEMS_Dp,RI_raw,RI_corr,GF_raw_160,GF_corr,UTC_Start_Time,UTC_Stop_Time,UTC_Mid_Time,GPS_Altitude
124,2024-05-28 15:36:10,2024-05-28 15:36:11,86.600000,160.0,1.369706,1.624879,2.111415,NaN,56170,56171,56170.5,172.000000
125,2024-05-28 15:36:11,2024-05-28 15:36:12,86.350000,160.0,1.369706,1.624879,1.735466,NaN,56171,56172,56171.5,172.000000
126,2024-05-28 15:36:12,2024-05-28 15:36:13,85.850000,160.0,1.369706,1.624879,2.108478,NaN,56172,56173,56172.5,172.000000
127,2024-05-28 15:36:13,2024-05-28 15:36:14,85.150000,160.0,1.369706,1.624879,2.293516,NaN,56173,56174,56173.5,172.000000
128,2024-05-28 15:36:14,2024-05-28 15:36:15,84.550000,160.0,1.369706,1.624879,1.735466,NaN,56174,56175,56174.5,172.428571
...,...,...,...,...,...,...,...,...,...,...,...,...
957,2024-05-28 17:04:18,2024-05-28 17:04:30,84.546154,160.0,1.403169,1.480070,2.293516,NaN,61458,61470,61464.0,NaN
966,2024-05-28 17:07:51,2024-05-28 17:07:57,84.200000,160.0,NaN,NaN,NaN,NaN,61671,61677,61674.0,NaN
967,2024-05-28 17:08:14,2024-05-28 17:08:23,85.050000,160.0,NaN,NaN,NaN,NaN,61694,61703,61698.5,NaN
1019,2024-05-28 17:21:35,2024-05-28 17:21:42,84.762500,160.0,NaN,NaN,NaN,NaN,62495,62502,62498.5,NaN


In [19]:
d_test[np.isfinite(d_test['GPS_Altitude'])]

,Start_Date_Time_UTC,Stop_Date_Time_UTC,RH,SEMS_Dp,RI_raw,RI_corr,GF_raw_160,GF_corr,UTC_Start_Time,UTC_Stop_Time,UTC_Mid_Time,GPS_Altitude
124,2024-05-28 15:36:10,2024-05-28 15:36:11,86.60,160.0,1.369706,1.624879,2.111415,NaN,56170,56171,56170.5,172.000000
125,2024-05-28 15:36:11,2024-05-28 15:36:12,86.35,160.0,1.369706,1.624879,1.735466,NaN,56171,56172,56171.5,172.000000
126,2024-05-28 15:36:12,2024-05-28 15:36:13,85.85,160.0,1.369706,1.624879,2.108478,NaN,56172,56173,56172.5,172.000000
127,2024-05-28 15:36:13,2024-05-28 15:36:14,85.15,160.0,1.369706,1.624879,2.293516,NaN,56173,56174,56173.5,172.000000
128,2024-05-28 15:36:14,2024-05-28 15:36:15,84.55,160.0,1.369706,1.624879,1.735466,NaN,56174,56175,56174.5,172.428571
129,2024-05-28 15:36:15,2024-05-28 15:36:16,84.35,160.0,1.369706,1.624879,1.732463,NaN,56175,56176,56175.5,173.000000
130,2024-05-28 15:36:16,2024-05-28 15:36:17,84.20,160.0,1.369706,1.624879,2.108478,NaN,56176,56177,56176.5,173.000000
131,2024-05-28 15:36:17,2024-05-28 15:36:18,84.20,160.0,1.369706,1.624879,1.735466,NaN,56177,56178,56177.5,172.666667
132,2024-05-28 15:36:18,2024-05-28 15:36:19,84.40,160.0,1.369706,1.624879,1.735466,NaN,56178,56179,56178.5,172.857143
133,2024-05-28 15:36:19,2024-05-28 15:36:20,84.60,160.0,1.369706,1.624879,1.926378,NaN,56179,56180,56179.5,173.000000


In [16]:
d_test

,Start_Date_Time_UTC,Stop_Date_Time_UTC,RH,SEMS_Dp,RI_raw,RI_corr,GF_raw_160,GF_corr,UTC_Start_Time,UTC_Stop_Time,UTC_Mid_Time,GPS_Altitude
124,2024-05-28 15:36:10,2024-05-28 15:36:11,86.600000,160.0,1.369706,1.624879,2.111415,NaN,56170,56171,56170.5,172.000000
125,2024-05-28 15:36:11,2024-05-28 15:36:12,86.350000,160.0,1.369706,1.624879,1.735466,NaN,56171,56172,56171.5,172.000000
126,2024-05-28 15:36:12,2024-05-28 15:36:13,85.850000,160.0,1.369706,1.624879,2.108478,NaN,56172,56173,56172.5,172.000000
127,2024-05-28 15:36:13,2024-05-28 15:36:14,85.150000,160.0,1.369706,1.624879,2.293516,NaN,56173,56174,56173.5,172.000000
128,2024-05-28 15:36:14,2024-05-28 15:36:15,84.550000,160.0,1.369706,1.624879,1.735466,NaN,56174,56175,56174.5,172.428571
...,...,...,...,...,...,...,...,...,...,...,...,...
957,2024-05-28 17:04:18,2024-05-28 17:04:30,84.546154,160.0,1.403169,1.480070,2.293516,NaN,61458,61470,61464.0,NaN
966,2024-05-28 17:07:51,2024-05-28 17:07:57,84.200000,160.0,NaN,NaN,NaN,NaN,61671,61677,61674.0,NaN
967,2024-05-28 17:08:14,2024-05-28 17:08:23,85.050000,160.0,NaN,NaN,NaN,NaN,61694,61703,61698.5,NaN
1019,2024-05-28 17:21:35,2024-05-28 17:21:42,84.762500,160.0,NaN,NaN,NaN,NaN,62495,62502,62498.5,NaN


In [7]:
r_dps[0]

,Start_Date_Time_UTC,Stop_Date_Time_UTC,RH,SEMS_Dp,RI_raw,RI_corr,GF_raw_160,GF_corr,UTC_Start_Time,UTC_Stop_Time,UTC_Mid_Time
124,2024-05-28 15:36:10,2024-05-28 15:36:11,86.600000,160.0,1.369706,1.624879,2.111415,NaN,56170,56171,56170.5
125,2024-05-28 15:36:11,2024-05-28 15:36:12,86.350000,160.0,1.369706,1.624879,1.735466,NaN,56171,56172,56171.5
126,2024-05-28 15:36:12,2024-05-28 15:36:13,85.850000,160.0,1.369706,1.624879,2.108478,NaN,56172,56173,56172.5
127,2024-05-28 15:36:13,2024-05-28 15:36:14,85.150000,160.0,1.369706,1.624879,2.293516,NaN,56173,56174,56173.5
128,2024-05-28 15:36:14,2024-05-28 15:36:15,84.550000,160.0,1.369706,1.624879,1.735466,NaN,56174,56175,56174.5
...,...,...,...,...,...,...,...,...,...,...,...
957,2024-05-28 17:04:18,2024-05-28 17:04:30,84.546154,160.0,1.403169,1.480070,2.293516,NaN,61458,61470,61464.0
966,2024-05-28 17:07:51,2024-05-28 17:07:57,84.200000,160.0,NaN,NaN,NaN,NaN,61671,61677,61674.0
967,2024-05-28 17:08:14,2024-05-28 17:08:23,85.050000,160.0,NaN,NaN,NaN,NaN,61694,61703,61698.5
1019,2024-05-28 17:21:35,2024-05-28 17:21:42,84.762500,160.0,NaN,NaN,NaN,NaN,62495,62502,62498.5
